In [ ]:
import sys
sys.path.insert(0, '/Users/matthewashman/github/MasterProject2018')

# Import necessary modules. Set settings. Import data.
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import pywt
import math
from IPython.display import HTML


# For feature extraction
from scipy.interpolate import CubicSpline      # for warping
from statsmodels.robust import mad
from tsfresh.feature_extraction import feature_calculators
from FeatureExtraction.feature_tools import detect_peaks

# Miscelaneous
from IPython.display import display, clear_output
import pdb

plt.style.use('default')

training_data = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/training_data.pkl')
validation_data =pd.read_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/validation_data.pkl')
test_data = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/test_data.pkl')

### Create Multiple Sets of Augmented Data

In [ ]:
warp_degrees = [0.1, 0.2, 0.3, 0.4]
training_data['Augmented']=0
validation_data['Augmented']=0
test_data['Augmented']=0

In [4]:
# %matplotlib qt 

# Use for storing augmented_rows in the form of dicts
for degree in warp_degrees:
    augmented_list = []
    for _, row in training_data[(training_data['Label']=='2')].iterrows():

        augmented_data = augment_fractionation(row['Data'], 5, warp_degree=degree, warp_cycles=4, 
                                               only_mag=True, debug=False)

#         fig, axes = plt.subplots(nrows=6, ncols=1, figsize=(16,9))
#         axes[0].plot(row['Data'])
        for i, data in enumerate(augmented_data):
            augmented_row = {}
            # Fill in the other column values
            for col, value in row.iteritems():
                augmented_row[col] = value

            augmented_row['Data'] = data
            augmented_row['Augmented']=1
            augmented_list.append(augmented_row)
            
#             axes[i+1].plot(data, 'C1')
#             axes[i+1].plot(row['Data'], alpha=0.3, linestyle='--')
        
#         axes[0].set_title(row['Type'] + row['Patient'] + ' ' + row['Coupling Interval'] + 
#                           ' Warp Degree: ' + str(degree))
#         plt.draw()
#         plt.waitforbuttonpress()
#         plt.close()    
    
    augmented_df = pd.DataFrame(augmented_list)
    
    file_name = 'augmented_training_data_' + str(np.int(degree*10)) +'_2.pkl'
    augmented_df.to_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/' + file_name)

In [5]:
# %matplotlib qt 

# Use for storing augmented_rows in the form of dicts
for degree in warp_degrees:
    augmented_list = []
    for _, row in validation_data[(validation_data['Label']=='2')].iterrows():

        augmented_data = augment_fractionation(row['Data'], 5, warp_degree=degree, warp_cycles=4, 
                                               only_mag=True, debug=False)

#         fig, axes = plt.subplots(nrows=6, ncols=1, figsize=(16,9))
#         axes[0].plot(row['Data'])
        for i, data in enumerate(augmented_data):
            augmented_row = {}
            # Fill in the other column values
            for col, value in row.iteritems():
                augmented_row[col] = value

            augmented_row['Data'] = data
            augmented_row['Augmented']=1
            augmented_list.append(augmented_row)
            
#             axes[i+1].plot(data, 'C1')
#             axes[i+1].plot(row['Data'], alpha=0.3, linestyle='--')
        
#         axes[0].set_title(row['Type'] + row['Patient'] + ' ' + row['Coupling Interval'] + 
#                           ' Warp Degree: ' + str(degree))
#         plt.draw()
#         plt.waitforbuttonpress()
#         plt.close()    
    
    augmented_df = pd.DataFrame(augmented_list)
    
    file_name = 'augmented_validation_data_' + str(np.int(degree*10)) +'_2.pkl'
    augmented_df.to_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/' + file_name)

In [ ]:
# %matplotlib qt 

# Use for storing augmented_rows in the form of dicts
for degree in warp_degrees:
    augmented_list = []
    for _, row in test_data[(test_data['Label']=='2')].iterrows():

        augmented_data = augment_fractionation(row['Data'], 5, warp_degree=degree, warp_cycles=4, 
                                               only_mag=True, debug=False)

#         fig, axes = plt.subplots(nrows=6, ncols=1, figsize=(16,9))
#         axes[0].plot(row['Data'])
        for i, data in enumerate(augmented_data):
            augmented_row = {}
            # Fill in the other column values
            for col, value in row.iteritems():
                augmented_row[col] = value

            augmented_row['Data'] = data
            augmented_row['Augmented']=1
            augmented_list.append(augmented_row)
            
#             axes[i+1].plot(data, 'C1')
#             axes[i+1].plot(row['Data'], alpha=0.3, linestyle='--')
        
#         axes[0].set_title(row['Type'] + row['Patient'] + ' ' + row['Coupling Interval'] + 
#                           ' Warp Degree: ' + str(degree))
#         plt.draw()
#         plt.waitforbuttonpress()
#         plt.close()    
    
    augmented_df = pd.DataFrame(augmented_list)
    
    file_name = 'augmented_test_data_' + str(np.int(degree*10)) +'.pkl'
    augmented_df.to_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/' + file_name)

In [ ]:
def augment_fractionation(x, N, only_mag=False, warp_degree=0.2, warp_cycles=4, debug=False):
    # Jittering
    def jitter(x, sigma=0.05):
        my_noise = np.random.normal(loc=0, scale=sigma, size=len(x))
        return x+my_noise

    # Scaling
    def scaling(x, sigma=0.1):
        scaling_factor = np.random.normal(loc=1.0, scale=sigma)
        return x*scaling_factor

    def generate_random_curves(x, sigma=0.2, knot=50):
        xx = np.arange(0,len(x), (len(x)-1)/(knot+1))
        yy = np.random.normal(loc=1.0, scale=sigma, size=knot+2)

        x_range = np.arange(len(x))
        cs = CubicSpline(xx, yy, bc_type='natural')
        
        # Only allow overall scaling of max amplitude between 0.75 and 1.5
        random_curve = cs(x_range)
        if max(abs(random_curve))>1.5:
            random_curve *= 1.2/max(abs(random_curve))
        elif max(abs(random_curve))<0.5:
            random_curve *= 0.75/max(abs(random_curve))
        
        return np.array(cs(x_range))

    # Magnitude warping
    def magwarp(x, sigma):
        cs = generate_random_curves(x, sigma, knot=35)
        # Blend cs curve towards 1 at both ends
        # tau = 10
        # delta = int(np.floor(len(x)/2))
        # for i in range(delta):
        #     cs[i] = (1-np.exp(-i/tau))*cs[i] + np.exp(-i/tau)*0
        #     cs[-(i+1)] = (1-np.exp(-i/tau))*cs[-(i+1)] + np.exp(-i/tau)*0

        return x*cs

    def distort_timesteps(x, sigma=0.2):
        tt = generate_random_curves(x, sigma, knot=7) # Regard these samples aroun 1 as time intervals
        tt_cum = np.cumsum(tt)        # Add intervals to make a cumulative graph
        # Make the last value to have X.shape[0]
        t_scale = (len(x)-1)/tt_cum[-1]
        tt_cum = tt_cum*t_scale
        return tt_cum

    # Time warping
    def timewarp(x, sigma=0.2):
        tt_new = distort_timesteps(x, sigma)
        x_new = np.zeros(len(x))
        x_range = np.arange(len(x))
        x_new = np.interp(x_range, tt_new, x)
        return x_new

    # Rotation
    def rotation(x):
        axis = np.random.uniform(low=-1, high=1, size=1)
        angle = np.random.uniform(low=-np.pi, high=np.pi)
        return np.matmul(x , axangle2mat(axis,angle))


    # x = x/max(abs(x))   # Normalise
    x_aug = np.zeros([N, len(x)])

    # Create N augmented examples
    if only_mag == False:
        for i in range(0,N):
            x_aug[i,:] = x[:]
            for j in range(warp_cycles):
                x_aug[i,:] = timewarp(x_aug[i,:], warp_degree)
                x_aug[i,:] = magwarp(x_aug[i,:], warp_degree)
    else:
         for i in range(0,N):
            x_aug[i,:] = x[:]
            for j in range(warp_cycles):
                x_aug[i,:] = magwarp(x_aug[i,:], warp_degree)


    if ((debug==True) & (N>3)):
        fig, [ax1, ax2, ax3, ax4, ax5] = plt.subplots(nrows=5, ncols=1, sharex=True, figsize=(4,6))
        ax1.plot(x)
        ax1.axis('off')
        for i, ax in enumerate([ax2, ax3, ax4, ax5]):
            ax.plot(x/max(abs(x)), '--', alpha=0.5)
            ax.hold(True)
            ax.plot(x_aug[i,:]/max(abs(x_aug[i,:])))
            ax.axis('off')

        plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
        plt.draw()
        plt.waitforbuttonpress()
        plt.close(fig)

    return x_aug